# Chargement de la base de données

In [1]:
import os
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
path = os.getcwd()
print(path)

/Users/pereiradearaujo/Documents/Scolarité/ENSEEIHT/Stage2A


In [3]:
path_directory = path + "/BD_71/"
print(path_directory)

/Users/pereiradearaujo/Documents/Scolarité/ENSEEIHT/Stage2A/BD_71/


In [4]:
IMG_SIZE = 224
train_ds = keras.utils.image_dataset_from_directory(
    directory=path_directory,
    labels='inferred',
    label_mode='categorical',
    shuffle = False,
    batch_size=16,
    image_size=(IMG_SIZE, IMG_SIZE))

Found 111979 files belonging to 71 classes.


2022-06-23 09:18:13.678958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
class_names = train_ds.class_names
print(class_names)
nb_classes = len(class_names)
print(nb_classes)

['Amegilla quadrifasciata', 'Andrena agilissima', 'Andrena bicolor', 'Andrena cineraria', 'Andrena clarkella', 'Andrena denticulata', 'Andrena flavipes', 'Andrena florea', 'Andrena fulva', 'Andrena gravida', 'Andrena haemorrhoa', 'Andrena hattorfiana', 'Andrena nigroaenea', 'Andrena nitida', 'Andrena nycthemera', 'Andrena thoracica', 'Andrena vaga', 'Andrena ventralis', 'Anthidiellum strigatum', 'Anthidium florentinum', 'Anthidium manicatum', 'Anthidium oblongatum', 'Anthidium septemspinosum', 'Anthophora bimaculata', 'Anthophora furcata', 'Anthophora plumipes', 'Apis mellifera', 'Bombus argillaceus', 'Bombus bohemicus', 'Bombus campestris', 'Bombus hortorum', 'Bombus humilis', 'Bombus hypnorum', 'Bombus lapidarius', 'Bombus lucorum', 'Bombus muscorum', 'Bombus pascuorum', 'Bombus pratorum', 'Bombus ruderatus', 'Bombus rupestris', 'Bombus sylvarum', 'Bombus terrestris lusitanicus', 'Bombus vestalis', 'Ceratina cucurbitina', 'Chelostoma florisomne', 'Colletes cunicularius', 'Colletes he

# Chargement du modèle

In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
import tensorflow as tf

In [7]:
# Création de l'architecture du modèle à utiliser

conv_base = keras.applications.resnet.ResNet101(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling=None,
    classes=nb_classes,
)

model = keras.Sequential(
    [
        conv_base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(nb_classes, kernel_regularizer=regularizers.L2(1e-4), activation='softmax')
    ]
)

model.load_weights(path + "/Poids/ResNet101/PoidsNormalisation67-5/best_model_seq_normalisation")

model.compile(optimizers.SGD(learning_rate=1e-3, momentum=0.9), loss='categorical_crossentropy', metrics=['categorical_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [55]:
import numpy as np
import time

start_time = time.time()
x = np.zeros((3979, IMG_SIZE, IMG_SIZE, 3))
y = np.zeros((3979, nb_classes))

ind_data = 0
lb = 108000
up = 111979
for bx, by in train_ds.as_numpy_iterator():
    if (ind_data < lb):
        ind_data += bx.shape[0]
    elif (ind_data >= lb and ind_data < up):
        x[ind_data-lb:ind_data-lb+bx.shape[0]] = bx
        y[ind_data-lb:ind_data-lb+bx.shape[0]] = by
        ind_data += bx.shape[0]
    elif (ind_data >= up):
        break
print("--- %s seconds ---" % (time.time() - start_time))
x = tf.keras.applications.resnet.preprocess_input(x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 84.33417582511902 seconds ---
--- 109.45438003540039 seconds ---


In [56]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 0


In [57]:
y_pred = model.predict(x)

125/125 [==============================] - 438s 3s/step


In [58]:
import tensorflow
from tensorflow.keras.metrics import Accuracy

In [59]:
test_accuracy = Accuracy()

prediction = tensorflow.argmax(y_pred, axis=1, output_type=tensorflow.int32)
print(prediction)
test_accuracy(prediction, np.argmax(y, axis=1))

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

tf.Tensor([70 70 69 ... 70 70 69], shape=(3979,), dtype=int32)
Test set accuracy: 57.301%


In [16]:
print(y_pred)
print(y)

[[4.4251806e-01 1.8143341e-04 5.8528781e-06 ... 8.6697277e-05
  2.1548227e-05 3.4068096e-05]
 [8.2680207e-01 1.2071311e-03 3.8482509e-07 ... 7.4913277e-04
  2.4135059e-05 3.6425547e-05]
 [9.9998844e-01 6.8209566e-08 3.3370534e-10 ... 1.8006937e-09
  8.5927720e-10 2.1439923e-09]
 ...
 [1.9534891e-04 7.4555238e-08 1.1411799e-05 ... 8.2490942e-07
  3.3600802e-06 2.8131515e-07]
 [1.5665959e-05 3.8423551e-07 1.4619975e-05 ... 9.6659228e-08
  6.6012376e-08 2.5060583e-08]
 [1.5786018e-04 3.8614563e-11 3.9284649e-07 ... 5.8492466e-10
  7.4974132e-10 1.1052712e-11]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
files = train_ds.file_paths

In [61]:
files[24000]

'/Users/pereiradearaujo/Documents/Scolarité/ENSEEIHT/Stage2A/BD_71/Apis mellifera/Apis mellifera18811.jpg'

# Création d'un DataFrame pour récupérer les prédictions

In [62]:
import pandas as pd
import os

In [63]:
# noms des fichiers et noms des classes
file_paths = train_ds.file_paths
class_names = train_ds.class_names

In [64]:
def save_pred_df(df, y_pred, y, starting_row=0):
    """
    Sauvegarder les informations d'une predictions dans un dataframe.
    
    Arguments :
    - df : le dataframe où on ajoute les informations : doit contenir
    species_name, species_num, ...
    - y_pred : la prédiction
    """
    
    nb_rows = y_pred.shape[0]
    
    for i in range(nb_rows):
        # nom de l'espèce
        class_num = np.argmax(y[i])
        class_name = class_names[class_num]

        # nom de l'image et path de l'image
        image_name = os.path.basename(files[i + starting_row])
        relative_image_path = "BD_71/" + class_name + "/" + image_name

        # proba associée à la ground truth
        probability = y_pred[i, class_num]
        predicted_species_num = np.argmax(y_pred[i])
        predicted_species_name = class_names[predicted_species_num]

        # Ajout des données au DataFrame
        row_to_add = {"species_name" : class_name,
                      "species_num" : class_num,
                      "predicted_species_name" : predicted_species_name, 
                      "predicted_species_num" : predicted_species_num,
                      "good_pred_probability" : probability,
                      "img_name" : image_name, 
                      "img_path" : relative_image_path}
        df = df.append(row_to_add, ignore_index=True)
        
    return df

In [18]:
#dic = {"species_name" : [], "species_num" : [], "predicted_species_name" : [], "predicted_species_num" : [], "good_pred_probability" : [], "img_name" : [], "img_path" : []}
#df = pd.DataFrame(dic)
#display(df)

In [23]:
path = os.getcwd()
df = pd.read_csv(path + "/predictions7.csv")
display(df)

,Unnamed: 0,species_name,species_num,predicted_species_name,predicted_species_num,good_pred_probability,img_name,img_path
0,0,Amegilla quadrifasciata,0.0,Andrena vaga,16.0,0.442518,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
1,1,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.826802,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
2,2,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999988,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
3,3,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999540,Amegilla quadrifasciata10653.jpg,BD_71/Amegilla quadrifasciata/Amegilla quadrif...
4,4,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999822,Amegilla quadrifasciata12804.jpg,BD_71/Amegilla quadrifasciata/Amegilla quadrif...
...,...,...,...,...,...,...,...,...
83995,83995,Bombus pascuorum,36.0,Bombus pascuorum,36.0,0.991762,Bombus pascuorum64406.jpg,BD_71/Bombus pascuorum/Bombus pascuorum64406.jpg
83996,83996,Bombus pascuorum,36.0,Bombus pascuorum,36.0,0.987107,Bombus pascuorum64419.jpg,BD_71/Bombus pascuorum/Bombus pascuorum64419.jpg
83997,83997,Bombus pascuorum,36.0,Bombus humilis,31.0,0.010117,Bombus pascuorum64424.jpeg,BD_71/Bombus pascuorum/Bombus pascuorum64424.jpeg
83998,83998,Bombus pascuorum,36.0,Bombus pascuorum,36.0,0.887520,Bombus pascuorum64434.jpeg,BD_71/Bombus pascuorum/Bombus pascuorum64434.jpeg


In [24]:
df.drop(columns='Unnamed: 0', inplace=True)
display(df)

,species_name,species_num,predicted_species_name,predicted_species_num,good_pred_probability,img_name,img_path
0,Amegilla quadrifasciata,0.0,Andrena vaga,16.0,0.442518,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
1,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.826802,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
2,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999988,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
3,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999540,Amegilla quadrifasciata10653.jpg,BD_71/Amegilla quadrifasciata/Amegilla quadrif...
4,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999822,Amegilla quadrifasciata12804.jpg,BD_71/Amegilla quadrifasciata/Amegilla quadrif...
...,...,...,...,...,...,...,...
83995,Bombus pascuorum,36.0,Bombus pascuorum,36.0,0.991762,Bombus pascuorum64406.jpg,BD_71/Bombus pascuorum/Bombus pascuorum64406.jpg
83996,Bombus pascuorum,36.0,Bombus pascuorum,36.0,0.987107,Bombus pascuorum64419.jpg,BD_71/Bombus pascuorum/Bombus pascuorum64419.jpg
83997,Bombus pascuorum,36.0,Bombus humilis,31.0,0.010117,Bombus pascuorum64424.jpeg,BD_71/Bombus pascuorum/Bombus pascuorum64424.jpeg
83998,Bombus pascuorum,36.0,Bombus pascuorum,36.0,0.887520,Bombus pascuorum64434.jpeg,BD_71/Bombus pascuorum/Bombus pascuorum64434.jpeg


In [46]:
print(files[111978])

/Users/pereiradearaujo/Documents/Scolarité/ENSEEIHT/Stage2A/BD_71/Xylocopa violacea/xylvio.jpg


In [65]:
df = save_pred_df(df, y_pred, y, starting_row = 108000)

In [66]:
display(df)

,species_name,species_num,predicted_species_name,predicted_species_num,good_pred_probability,img_name,img_path
0,Amegilla quadrifasciata,0.0,Andrena vaga,16.0,0.442518,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
1,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.826802,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
2,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999988,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,BD_71/Amegilla quadrifasciata/2019-08-01 Lumin...
3,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999540,Amegilla quadrifasciata10653.jpg,BD_71/Amegilla quadrifasciata/Amegilla quadrif...
4,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999822,Amegilla quadrifasciata12804.jpg,BD_71/Amegilla quadrifasciata/Amegilla quadrif...
...,...,...,...,...,...,...,...
111974,Xylocopa violacea,70.0,Xylocopa valga,69.0,0.176861,xylocopa_2301.jpg,BD_71/Xylocopa violacea/xylocopa_2301.jpg
111975,Xylocopa violacea,70.0,Heriades truncorum,51.0,0.000255,xylocopa_cadre.jpg,BD_71/Xylocopa violacea/xylocopa_cadre.jpg
111976,Xylocopa violacea,70.0,Xylocopa violacea,70.0,0.615408,xylocopa_violace.jpg,BD_71/Xylocopa violacea/xylocopa_violace.jpg
111977,Xylocopa violacea,70.0,Xylocopa violacea,70.0,0.840239,xylocope.jpg,BD_71/Xylocopa violacea/xylocope.jpg


In [43]:
#for i in range(12000, 24000):
#    image_name = os.path.basename(files[i])
#    class_name = "Apis mellifera"
#    relative_image_path = "BD_71/" + class_name + "/" + image_name
#    df.at[i, "img_name"] = image_name
#    df.at[i, "img_path"] = relative_image_path

In [67]:
#path = os.getcwd()
#df.drop(columns="Unnamed: 0", inplace=True)
#display(df)
df.to_csv(path + "/predictions71.csv")